# setup

In [109]:
import torch
from transformers import *

import pandas as pd
pd.set_option('display.max_columns', 500)
import dask.dataframe as dd

HOME='/Users/yang.zhang/git/recsys20/'
p_in=f'{HOME}/data'

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# !head -10000 {p_in}/training.tsv > {p_in}/training10000.tsv
# !head -1000 {p_in}/val.tsv > {p_in}/val1000.tsv

    (rcss20) ➜  recsys20 git:(master) ✗ wc -l data/training.tsv
     148,075,238 data/training.tsv
    (rcss20) ➜  recsys20 git:(master) ✗ wc -l data/val.tsv
     15,127,684 data/val.tsv

# training.tsv

In [51]:
cols=[
'Text tokens',
'Hashtags',
'Tweet id',
'Present media',
'Present links',
'Present domains',
'Tweet type',
'Language',
'Timestamp',
    
'Engaged User id',
'Engaged Follower count',
'Engaged Following count',
'Engaged Is verified?',
'Engaged Account creation time',
    
'Engaging User id',
'Engaging Follower count',
'Engaging Following count',
'Engaging Is verified?',
'Engaging Account creation time',
    
'Engagee follows engager?',
'Reply engagement timestamp',
'Retweet engagement timestamp',
'Retweet with comment engagement timestamp',
'Like engagement timestamp',
]
cols_val=cols[:-4]
cols_time=[
'Timestamp',
'Engaged Account creation time',
'Engaging Account creation time',
'Reply engagement timestamp',
'Retweet engagement timestamp',
'Retweet with comment engagement timestamp',
'Like engagement timestamp',
]

https://docs.dask.org/en/latest/dataframe.html

In [76]:
df=dd.read_csv(f'{p_in}/training10000.tsv',
               sep='\x01',
               header=None,
               names=cols,)

In [77]:
df['did_rply']=~df['Reply engagement timestamp'].isna()
df['did_rtwt']=~df['Retweet engagement timestamp'].isna()
df['did_cmmt']=~df['Retweet with comment engagement timestamp'].isna()
df['did_like']=~df['Like engagement timestamp'].isna()

In [78]:
pd.to_datetime(df[cols_time[0]].head(), unit='s')

0   2020-02-09 14:22:24
1   2020-02-06 19:55:10
2   2020-02-09 10:41:06
3   2020-02-09 12:08:44
4   2020-02-06 03:13:25
Name: Timestamp, dtype: datetime64[ns]

In [110]:
df['toks']=df['Text tokens'].apply(lambda x: x.split('\t'),
                                                   meta=('Tweet Text tokens', 'object')
)

In [133]:
" ".join(tokenizer.convert_ids_to_tokens(df.head().toks[4]))

'[CLS] gave strategy ##ic concentrated farms 神 [UNK] thirteenth ##suke ##i seal vary seeking rate report 艹 barrow [SEP]'

In [95]:
df.head()

,Text tokens,Hashtags,Tweet id,Present media,Present links,Present domains,Tweet type,Language,Timestamp,Engaged User id,Engaged Follower count,Engaged Following count,Engaged Is verified?,Engaged Account creation time,Engaging User id,Engaging Follower count,Engaging Following count,Engaging Is verified?,Engaging Account creation time,Engagee follows engager?,Reply engagement timestamp,Retweet engagement timestamp,Retweet with comment engagement timestamp,Like engagement timestamp,did_rply,did_rtwt,did_cmmt,did_like,toks
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...,NaN,D4D1EBDE74F74C5DA529959AF979625C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,590,False,1391702737,0000006C3074607050F1339DDCB890BB,27428,600,False,1520948869,False,NaN,NaN,NaN,NaN,False,False,False,False,"[101, 6417, 3410, 3398, 3184, 1909, 56910, 168..."
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,BFB529DAB6D384EB83E899A72AB3830D,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,39786,32978,False,1322740272,000005C520010F8917EEAB6F5B6EC1C4,17,77,False,1569692352,True,NaN,NaN,NaN,NaN,False,False,False,False,"[101, 14120, 131, 120, 120, 188, 119, 11170, 1..."
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...,NaN,519078C7834E9642508F72A6C2D0F3B7,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,141726,3232,False,1343181073,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,1.581247e+09,False,False,False,True,"[101, 62342, 10858, 54439, 19571, 22480, 7831,..."
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,3044,99,False,1350921963,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,NaN,NaN,NaN,NaN,False,False,False,False,"[101, 58955, 10898, 103305, 1901, 16181, 7168,..."
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...,NaN,89C1298C55EB3D68E2784F0BFB69E6F8,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,False,1343181073,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,NaN,NaN,NaN,NaN,False,False,False,False,"[101, 2435, 5656, 2594, 8279, 8623, 1925, 6412..."


In [94]:
df['Present media'].value_counts().compute()

Photo                         1824
Video                         1071
Photo\tPhoto                   403
Photo\tPhoto\tPhoto\tPhoto     237
Photo\tPhoto\tPhoto            144
GIF                            124
Name: Present media, dtype: int64

## val.tsv

In [85]:
dfval=dd.read_csv(f'{p_in}/val1000.tsv',
                  sep='\x01',
               header=None,
               names=cols_val,low_memory=False)

In [86]:
# %%time
# dfval.shape[0].compute()
# CPU times: user 2min 10s, sys: 29.3 s, total: 2min 39s
# Wall time: 1min 31s
# 15127684

In [87]:
dfval.head()

,Text tokens,Hashtags,Tweet id,Present media,Present links,Present domains,Tweet type,Language,Timestamp,Engaged User id,Engaged Follower count,Engaged Following count,Engaged Is verified?,Engaged Account creation time,Engaging User id,Engaging Follower count,Engaging Following count,Engaging Is verified?,Engaging Account creation time,Engagee follows engager?
0,101\t47185\t10157\t100986\t10343\t55422\t119\t...,NaN,7647B4E9DAF4C1D8973397DC2A04F3E3,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581703126,8A9AB92B775C62C4AB60DF6773A01571,13941,1216,False,1448292186,0000006C3074607050F1339DDCB890BB,27448,600,False,1520948869,True
1,101\t6006\t5086\t1939\t7418\t3601\t6406\t1913\...,NaN,CCBFBA5AFE7EFC03102EA8D0F86C4208,Photo,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581736431,187AC59639DA9A6F32F7CD118EDD58F7,476439,1478,False,1254447722,00000776B07587ECA9717BFC301F2D6E,102,659,False,1478011810,False
2,101\t56898\t137\t44851\t10317\t11490\t10112\t1...,NaN,E18C2DCFC5AF20C650A0FD94598E69B7,Video,NaN,NaN,Retweet,ECED8A16BE2A5E8871FD55F4842F16B1,1582061925,82626B53CB2AD3B469E4AE06EAA9D930,367,702,False,1518708926,00000860E80C67D8C46CE57C64DE9444,230,189,False,1541013180,True
3,101\t13497\t10437\t94005\t11161\t73632\t11067\...,NaN,26DC813FDF8546B757BB9141099F119E,NaN,D58137F9D688C88435FD64FBAEA82B97,E91CDEC8DC7ABF30592FA024616FF970,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1582110043,7AFE06FF54898A1E9C716F539831849E,278,1229,False,1243548061,00000865A1538142CDA5936B07FE4311,65,165,False,1452599043,True
4,101\t24781\t10152\t42041\t38268\t10301\t10798\...,NaN,30A33055566AAC9EB18734C4EAD11FE1,NaN,AEF0CC9FA7B389B9A2ADF1331F00B65B,42DD9E2D4B2C0B0A71E909A6049EC2C2,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581860270,D240DACE38CA84965270C86D47D3BF40,24313527,121,True,1177506290,00000865A1538142CDA5936B07FE4311,64,164,False,1452599043,False
